# Goal

<h3 style="color:blue">assess the quality of summaries written by students</h3>
<h3 style="color:indigo">evaluate how well a student represents the main idea and details of a source text, as well as the clarity, precision, and fluency of the language used in the summary</h3>
<h3 style="color:red">Freely & publicly available external data is <b>allowed</b>, including pre-trained models</h3>
<h3>This is Multi-Output problem</h3>

### Use Hugging Face Library
### Use NLTK
### Use Tensorflow

In [29]:
import warnings
warnings.filterwarnings("ignore")

In [30]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import math
import subprocess
from tqdm import tqdm
import pickle

In [31]:
import tensorflow as tf

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, explained_variance_score, median_absolute_error

In [33]:
import transformers
from transformers import AutoTokenizer, TFBertModel

In [34]:
prompts_train = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv')
summaries_train = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv')
prompts_test = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv')
summaries_test = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv')

In [35]:
train = pd.merge(prompts_train, summaries_train, on='prompt_id')
test = pd.merge(prompts_test, summaries_test, on='prompt_id')

In [36]:
train.rename(columns = {'text' : 'summary'}, inplace=True)
test.rename(columns = {'text' : 'summary'}, inplace=True)

In [37]:
train.head(2)

,prompt_id,prompt_question,prompt_title,prompt_text,student_id,summary,content,wording
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,00791789cc1f,1 element of an ideal tragedy is that it shoul...,-0.210614,-0.471415
1,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,0086ef22de8f,The three elements of an ideal tragedy are: H...,-0.970237,-0.417058


In [38]:
train['summary'][0]

'1 element of an ideal tragedy is that it should be arranged on a complex plan.  Another element of an ideal tragedy is that it should only have one main issue. The last element of an ideal tragedy is that it should have a double thread plot and an opposite catastrophe for both good and bad.'

In [39]:
columns_needed = ["prompt_text", "summary"]

In [40]:
train_data = train[columns_needed]
test_data = test[columns_needed]

In [41]:
#from transformers import XLNetTokenizer, TFXLNetModel
#tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
#model = TFXLNetModel.from_pretrained('xlnet-base-cased', return_dict=True)

#from transformers import RobertaTokenizer, TFRobertaModel
#tokenizer = RobertaTokenizer.from_pretrained('roberta-base-cased')
#model = TFRobertaModel.from_pretrained('roberta-base-cased', return_dict=True)

from transformers import AutoTokenizer, TFBertModel
model = TFBertModel.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

### Next time use prepare_tf_dataset which is used to directly tokenize and data colat and
### make dataset compatible with tensorflow
####       https://huggingface.co/docs/transformers/v4.31.0/en/main_classes/model#transformers.TFPreTrainedModel.prepare_tf_dataset

In [42]:

def vectorize_dataframe(dataframe, col):
    vectors = []
    for text in tqdm(dataframe[col].tolist()):
        text_tokens = tokenizer(text, return_tensors="tf", padding='max_length', truncation=True)
        
        output = model(text_tokens)
        
        pooler_output = output.pooler_output

        vectors.append(pooler_output)
    return vectors
    

In [43]:
test_data['prompt_text_embedded'] = vectorize_dataframe(test_data, 'prompt_text')
test_data['summary_embedded'] = vectorize_dataframe(test_data, 'summary')

100%|██████████| 4/4 [00:06<00:00,  1.53s/it]


In [44]:

#with open("BERT_prompt_text_embeddings.pkl", "wb") as file:
#    pickle.dump(train_data['prompt_text_embeddings'], file)

In [45]:
with open("/kaggle/input/embeddings/BERT_prompt_text_embeddings.pkl", "rb") as file:
    train_data['prompt_text_embedded'] = pickle.load(file)
    
with open("/kaggle/input/embeddings/BERT_summary_embeddings.pkl", "rb") as file:
    train_data['summary_embedded'] = pickle.load(file)

In [46]:
traning_set = train_data[['prompt_text_embedded', 'summary_embedded']]
testing_set = test_data[['prompt_text_embedded', 'summary_embedded']]

### Take average of embeddings  [Not required, just checking]

In [47]:
target1 = np.array(train['content'])
target1 = target1.astype('float32')

target2 = np.array(train['wording'])
target2 = target2.astype('float32')

#target = (target1, target2)

In [48]:
def convert_tensor_to_numpy(tensor):
        return np.array(tensor, dtype='float32')

traning_set = traning_set.applymap(convert_tensor_to_numpy)
testing_set = testing_set.applymap(convert_tensor_to_numpy)

In [49]:
def prepare_dataset(dataset):
    # Flatten the nested arrays in the DataFrame
    dataset['prompt_text_embedded'] = dataset['prompt_text_embedded'].apply(lambda x: x.flatten())
    dataset['summary_embedded'] = dataset['summary_embedded'].apply(lambda x: x.flatten())
    
    feature1 = np.array(dataset['prompt_text_embedded'].tolist())
    feature2 = np.array(dataset['summary_embedded'].tolist())
    
    features = np.concatenate((feature1, feature2), axis=1)
    
    return features

In [50]:
features = prepare_dataset(traning_set)

In [51]:
features_for_test = prepare_dataset(testing_set)

In [52]:
#X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [53]:
from tensorflow.keras.layers import Dense, Input, Flatten

In [54]:
# Define the model architecture
model_content = tf.keras.Sequential([
      tf.keras.layers.Dense(128, activation='linear'),
      tf.keras.layers.Dense(64, activation='linear'),
      tf.keras.layers.Dense(32, activation='linear'),
      tf.keras.layers.Dense(1, activation='linear')
])

model_wording = tf.keras.Sequential([
      tf.keras.layers.Dense(128, activation='linear'),
      tf.keras.layers.Dense(64, activation='linear'),
      tf.keras.layers.Dense(32, activation='linear'),
      tf.keras.layers.Dense(1, activation='linear')
])

In [55]:
model_content.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])
model_wording.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])

In [56]:
# Train your model using model.fit()
history1 = model_content.fit(features, target1, epochs=300)

Epoch 1/50
224/224 [==============================] - 2s 4ms/step - loss: 1.7892 - mae: 0.8007 - mse: 1.7892
Epoch 2/50
224/224 [==============================] - 1s 5ms/step - loss: 0.4477 - mae: 0.5185 - mse: 0.4477
Epoch 3/50
224/224 [==============================] - 1s 4ms/step - loss: 0.4169 - mae: 0.5004 - mse: 0.4169
Epoch 4/50
224/224 [==============================] - 1s 4ms/step - loss: 0.4131 - mae: 0.4994 - mse: 0.4131
Epoch 5/50
224/224 [==============================] - 1s 4ms/step - loss: 0.3718 - mae: 0.4705 - mse: 0.3718
Epoch 6/50
224/224 [==============================] - 1s 5ms/step - loss: 0.3808 - mae: 0.4804 - mse: 0.3808
Epoch 7/50
224/224 [==============================] - 1s 4ms/step - loss: 0.3750 - mae: 0.4751 - mse: 0.3750
Epoch 8/50
224/224 [==============================] - 1s 4ms/step - loss: 0.3665 - mae: 0.4695 - mse: 0.3665
Epoch 9/50
224/224 [==============================] - 1s 4ms/step - loss: 0.3784 - mae: 0.4768 - mse: 0.3784
Epoch 10/50
224/224

In [57]:
history2 = model_wording.fit(features, target2, epochs=300)

Epoch 1/50
224/224 [==============================] - 2s 5ms/step - loss: 4.1042 - mae: 1.0876 - mse: 4.1042
Epoch 2/50
224/224 [==============================] - 1s 5ms/step - loss: 0.6168 - mae: 0.6191 - mse: 0.6168
Epoch 3/50
224/224 [==============================] - 1s 5ms/step - loss: 0.5837 - mae: 0.5993 - mse: 0.5837
Epoch 4/50
224/224 [==============================] - 1s 5ms/step - loss: 0.5740 - mae: 0.5936 - mse: 0.5740
Epoch 5/50
224/224 [==============================] - 1s 5ms/step - loss: 0.5445 - mae: 0.5787 - mse: 0.5445
Epoch 6/50
224/224 [==============================] - 1s 5ms/step - loss: 0.5472 - mae: 0.5808 - mse: 0.5472
Epoch 7/50
224/224 [==============================] - 1s 5ms/step - loss: 0.5340 - mae: 0.5720 - mse: 0.5340
Epoch 8/50
224/224 [==============================] - 1s 5ms/step - loss: 0.5278 - mae: 0.5704 - mse: 0.5278
Epoch 9/50
224/224 [==============================] - 1s 5ms/step - loss: 0.5283 - mae: 0.5687 - mse: 0.5283
Epoch 10/50
224/224

In [58]:
evaluate_on_train_content = model_content.evaluate(features, target1)
evaluate_on_train_wording = model_wording.evaluate(features, target2)

224/224 [==============================] - 1s 2ms/step - loss: 0.4533 - mae: 0.5290 - mse: 0.4533


In [59]:
print('evaluate_on_train_content', evaluate_on_train_content)
print('evaluate_on_train_wording',evaluate_on_train_wording)

evaluate_on_train_content [0.28219616413116455, 0.40824583172798157, 0.28219616413116455]
evaluate_on_train_wording [0.4533208906650543, 0.529002845287323, 0.4533208906650543]


In [60]:
content_prediction = model_content.predict(features)
wording_prediction = model_wording.predict(features)

224/224 [==============================] - 0s 2ms/step


### Predict on test

In [73]:
test_pred_content = model_content.predict(features_for_test)
test_pred_wording = model_wording.predict(features_for_test)

1/1 [==============================] - 0s 28ms/step


## submission

In [78]:
test_pred_content = test_pred_content.reshape(-1)
test_pred_wording = test_pred_wording.reshape(-1)

In [79]:
submission = pd.DataFrame({
    'student_id' : test['student_id'],
    'content' : test_pred_content,
    'wording' : test_pred_wording
})

In [80]:
submission.to_csv('submission.csv', index=False)

In [81]:
submission.head()

,student_id,content,wording
0,000000ffffff,-2.076940,-1.874210
1,222222cccccc,-1.913587,-1.807409
2,111111eeeeee,-2.034903,-1.879025
3,333333dddddd,-1.982370,-1.836262
